In [1]:
# ignore all warning messages
import warnings
warnings.filterwarnings('ignore')
import os
import torch

%env ONEAPI_INSTALL=/opt/intel/oneapi

env: ONEAPI_INSTALL=/opt/intel/oneapi


In [2]:
from mmseg.apis import inference_model

def get_average_inference_time(model, data):
    """
    does a model warm up and times the model runtime
    """
    with torch.no_grad():
        # warm up
        for _ in range(10):
            inference_model(model, data)

        # measure
        import time
        start = time.time()
        for _ in range(10):
            output = inference_model(model, data)
        end = time.time()
        average_inference_time = (end-start)/10*1000
    
    return average_inference_time

In [3]:
from mmseg.apis import inference_model, init_model, show_result_pyplot
import mmcv

config_file = 'configs/stdc/stdc2_4xb12-80k_cityscapes-512x1024.py'
checkpoint_file = 'ckpt/stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a.pth'

# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cpu')

# test a single image and show the results
img = 'demo/demo.png'  # or img = mmcv.imread(img), which will only load it once
# result = inference_model(model, img)
infer_time = get_average_inference_time(model, img)
print('average time: %.2f ms' % infer_time)

# visualize the results in a new window
# show_result_pyplot(model, img, result, show=True)
# or save the visualization results to image files
# you can change the opacity of the painted segmentation map in (0, 1].
# show_result_pyplot(model, img, result, show=False, out_file='result.jpg', opacity=0.5)

Loads checkpoint by local backend from path: ckpt/stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a.pth
average time: 587.69 ms


In [4]:
from mmseg.apis import inference_model, init_model, show_result_pyplot
import mmcv
import torch
import numpy as np
import time
import os

T = 800 # number of inference samples

config_file = 'configs/stdc/stdc2_4xb12-80k_cityscapes-512x1024.py'
checkpoint_file = 'ckpt/stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a.pth'

imgs = []

filepath = './data/cityscapes/leftImg8bit/test'
extends = '.png'
filelist=[]
for root, dirnames, filenames in os.walk(filepath):
    for filename in filenames:
        if extends in filename:
            filelist.append(os.path.join(root,filename))

imgs_path = np.random.choice(filelist, T, replace=False)
for img_path in imgs_path:
    imgs.append(mmcv.imread(img_path))

    
# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cpu')

with torch.no_grad():
    # warm up
    for img in imgs[:20]:
        inference_model(model, img)

    # measure
    import time
    start = time.time()
    for img in imgs:
        output = inference_model(model, img)
    end = time.time()
    average_inference_time = (end-start)/T*1000

print('average time: %.2f ms' % average_inference_time)
    
# # visualize inference result
# ## random a palette with size 256x3
# palette = np.random.randint(0, 256, size=(256, 3))
# color_seg = np.zeros((seg.shape[0], seg.shape[1], 3), dtype=np.uint8)
# for label, color in enumerate(palette):
#     color_seg[seg == label, :] = color
# # convert to BGR
# color_seg = color_seg[..., ::-1]
# img = img * 0.5 + color_seg * 0.5
# img = img.astype(np.uint8)
# cv2.imwrite('output_segmentation.png', img)

Loads checkpoint by local backend from path: ckpt/stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a.pth
average time: 410.16 ms


In [4]:
#######################################################################################################################################
import neural_compressor # Intel® Neural Compressor is an open-source Python library that supports popular model compression techniques
#######################################################################################################################################

# check if Neural Compressor is above v2.0
assert float(neural_compressor.__version__) >= 2.0, "The below APIs work with Intel® Neural Compressor 2.0 and above" 

from neural_compressor.quantization import fit
from neural_compressor.config import PostTrainingQuantConfig, TuningCriterion, AccuracyCriterion

In [5]:
class MIou(object):
    def __init__(self, numClass=19):
        self.numClass = numClass
        self.confusionMatrix = torch.zeros((self.numClass,) * 2)  # 混淆矩阵（空）

        
    def update(self, imgPredict, imgLabel):
        assert imgPredict.shape == imgLabel.shape
        self.confusionMatrix += self.genConfusionMatrix(imgPredict, imgLabel)  # 得到混淆矩阵
        return self.confusionMatrix


    def genConfusionMatrix(self, imgPredict, imgLabel, ignore_labels=[]):  #
        # remove classes from unlabeled pixels in gt image and predict
        mask = (imgLabel >= 0) & (imgLabel < self.numClass)
        for IgLabel in ignore_labels:
            mask &= (imgLabel != IgLabel)
        label = self.numClass * imgLabel[mask] + imgPredict[mask]
        count = torch.bincount(label, minlength=self.numClass ** 2)
        confusionMatrix = count.view(self.numClass, self.numClass)
        # print(confusionMatrix)
        return confusionMatrix

    
    def reset(self):
        # clear preds and labels storage
        self.confusionMatrix = torch.zeros((self.numClass, self.numClass))

        
    def IntersectionOverUnion(self):
        # Intersection = TP Union = TP + FP + FN
        # IoU = TP / (TP + FP + FN)
        intersection = torch.diag(self.confusionMatrix)  # 取对角元素的值，返回列表
        union = torch.sum(self.confusionMatrix, axis=1) + torch.sum(self.confusionMatrix, axis=0) - torch.diag(
            self.confusionMatrix)  # axis = 1表示混淆矩阵行的值，返回列表； axis = 0表示取混淆矩阵列的值，返回列表
        IoU = intersection / union  # 返回列表，其值为各个类别的IoU
        return IoU
        
        
    def result(self):
        IoU = self.IntersectionOverUnion()
        mIoU = IoU[IoU<float('inf')].mean()# 求各类别IoU的平均
        return mIoU 

In [6]:
from neural_compressor.data import DataLoader, Datasets
from neural_compressor.config import PostTrainingQuantConfig
# from mmseg.datasets import CityscapesDataset
from torchvision.datasets import Cityscapes
from torch.utils.data import DataLoader
from mmengine.runner import Runner

# tuning_criterion = TuningCriterion(max_trials=5)
# accuracy_criterion = AccuracyCriterion(tolerable_loss=0.1)
# conf = PostTrainingQuantConfig(approach="dynamic", tuning_criterion=tuning_criterion, accuracy_criterion=accuracy_criterion)

# conf = PostTrainingQuantConfig()
# dataset = Datasets("pytorch")["dummy"](shape=(1, 512, 1024, 3))
# dataloader = DataLoader(framework="pytorch", dataset=dataset)

miou = MIou()
config = PostTrainingQuantConfig()

dataset = Cityscapes('./data/cityscapes', split='test', mode='fine',
                     target_type='semantic', transforms=)
test_dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=False)

q_model = fit(model, config, calib_dataloader=test_dataloader, eval_dataloader=test_dataloader, eval_metric=miou)

2023-10-07 23:48:20 [INFO] Start auto tuning.
2023-10-07 23:48:20 [INFO] Create evaluation function according to evaluation dataloader and metric                and Execute the tuning process.
2023-10-07 23:48:20 [INFO] Adaptor has 4 recipes.
2023-10-07 23:48:20 [INFO] 0 recipes specified by user.
2023-10-07 23:48:20 [INFO] 3 recipes require future tuning.
2023-10-07 23:48:20 [INFO] *** Initialize auto tuning
2023-10-07 23:48:20 [INFO] {
2023-10-07 23:48:20 [INFO]     'PostTrainingQuantConfig': {
2023-10-07 23:48:20 [INFO]         'AccuracyCriterion': {
2023-10-07 23:48:20 [INFO]             'criterion': 'relative',
2023-10-07 23:48:20 [INFO]             'higher_is_better': True,
2023-10-07 23:48:20 [INFO]             'tolerable_loss': 0.01,
2023-10-07 23:48:20 [INFO]             'absolute': None,
2023-10-07 23:48:20 [INFO]             'keys': <bound method AccuracyCriterion.keys of <neural_compressor.config.AccuracyCriterion object at 0x7facf4b89df0>>,
2023-10-07 23:48:20 [INFO]      

In [14]:
# test a single image and show the results
img = 'demo/demo.png'  # or img = mmcv.imread(img), which will only load it once
# result = inference_model(model, img)
infer_time = get_average_inference_time(q_model._model, img)
print('average time: %.2f ms' % infer_time)

average time: 533.78 ms


In [38]:
torch.save(q_model._model.state_dict(), 'ckpt/stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a-quan.pth')

In [3]:
###################################

In [ ]:
import neural_compressor as inc
print("neural_compressor version {}".format(inc.__version__))

import torch
print("torch {}".format(torch.__version__))

from neural_compressor.quantization import fit
from neural_compressor.config import PostTrainingQuantConfig, AccuracyCriterion, TuningCriterion, Options
from neural_compressor.data import DataLoader
from torchvision.datasets import Cityscapes


In [ ]:
batch_size = 4
fp32_model_file = 'ckpt/stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a.pth'
int8_model = 'ckpt/stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a-quan-static.pth'

config_file = 'configs/stdc/stdc2_4xb12-80k_cityscapes-512x1024.py'
checkpoint_file = 'ckpt/stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a.pth'
# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cpu')


if compare_ver(inc.__version__, "2.0")>=0:
    print(f"Compatible Intel Neural Compressor version detected : v{inc.__version__} ")
else:
    raise Exception(f"Installed Intel Neural Compressor version[v{inc.__version__}] is NOT compatible. Please upgrade to version 2.0 or higher.")

# train_dataset, test_dataset = fashion_mnist.download_dataset()
# calib_dataloader = DataLoader(framework='pytorch', dataset=train_dataset, batch_size = batch_size)
# eval_dataloader = DataLoader(framework='pytorch', dataset=test_dataset, batch_size = batch_size)

train_dataset = Cityscapes('./data/cityscapes', split='train', mode='fine', target_type='semantic')
test_dataset = Cityscapes('./data/cityscapes', split='test', mode='fine', target_type='semantic')
calib_dataloader = DataLoader(framework='pytorch', dataset=train_dataset, batch_size = batch_size)
eval_dataloader = DataLoader(framework='pytorch', dataset=test_dataset, batch_size = batch_size)

tuning_criterion = TuningCriterion(max_trials=100)
option = Options(workspace='./nc_workspace')
config = PostTrainingQuantConfig(approach="static", tuning_criterion=tuning_criterion,
accuracy_criterion = AccuracyCriterion(
  higher_is_better=True, 
  criterion='relative',  
  tolerable_loss=0.01  
  )
)
q_model = fit(
    model=model,
    conf=config,
    calib_dataloader=calib_dataloader,
    eval_dataloader=eval_dataloader
    )
    
q_model = auto_tune(fp32_model_file, batch_size)
q_model.save(int8_model)
print("Save int8 model to {}".format(int8_model))